In [1]:
import torch
from torch.profiler import profile, record_function, ProfilerActivity
import time
import os
import matplotlib.pyplot as plt
from datetime import datetime

class CustomPyTorchProfiler:
    def __init__(self, trace_name="default_trace", output_dir="output", activities=None):
        """Initializes the custom profiler with given configurations."""
        if activities is None:
            activities = [ProfilerActivity.CPU]  # Default to CPU for macOS

        self.trace_name = trace_name
        self.output_dir = output_dir
        self.activities = activities
        self.profiler = None
        self.start_time = None
        self.end_time = None
        self.create_output_dir()

    def create_output_dir(self):
        """Ensure the output directory exists."""
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)

    def start(self):
        """Start the profiling session."""
        self.start_time = time.time()
        self.profiler = profile(
            activities=self.activities,
            on_trace_ready=torch.profiler.tensorboard_trace_handler(self.output_dir),
            record_shapes=True,
            profile_memory=True,
            with_stack=True
        )
        self.profiler.__enter__()

    def stop(self):
        """Stop the profiling session and export summary."""
        self.end_time = time.time()
        self.profiler.__exit__(None, None, None)
        self.export_summary_report()

    def record_function(self, name, func):
        """Wrap a function with profiling."""
        with record_function(name):
            func()

    def get_summary(self):
        """Returns summary of the profiling session."""
        return {
            "trace_name": self.trace_name,
            "duration": self.end_time - self.start_time,
            "output_dir": self.output_dir
        }

    def log_summary(self):
        """Prints the profiling summary."""
        summary = self.get_summary()
        print("\n[Profiler Summary]")
        print(f"Trace Name     : {summary['trace_name']}")
        print(f"Total Duration : {summary['duration']:.4f} seconds")
        print(f"Output Dir     : {summary['output_dir']}")
        print(f"\nView it in TensorBoard:\n  tensorboard --logdir={summary['output_dir']}\n")

    def export_summary_report(self):
        """Exports Markdown and a chart summarizing the profiling session."""
        summary = self.get_summary()
        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Markdown report
        report_md = f"""# PyTorch Profiling Report

**Generated**: {now}  
**Trace Name**: `{summary['trace_name']}`  
**Total Duration**: `{summary['duration']:.4f} seconds`  
**Output Directory**: `{summary['output_dir']}`  

## How to View

Run this in your terminal: tensorboard --logdir={summary['output_dir']}

Then open [http://localhost:6006](http://localhost:6006)

## Notes

- CPU-only profiling (macOS doesn't support CUDA)
- View the *"Profiler"* tab in TensorBoard
"""

        # Save Markdown
        report_path = os.path.join(summary['output_dir'], f"{summary['trace_name']}_report.md")
        with open(report_path, "w") as f:
            f.write(report_md)
        print(f"[+] Markdown report saved: {report_path}")

        # Save duration chart
        fig, ax = plt.subplots()
        ax.bar(["Profile Duration"], [summary['duration']], color='skyblue')
        ax.set_ylabel("Seconds")
        ax.set_title(f"Trace: {summary['trace_name']}")
        fig.tight_layout()

        chart_path = os.path.join(summary['output_dir'], f"{summary['trace_name']}_duration_chart.png")
        plt.savefig(chart_path)
        plt.close()
        print(f"[+] Duration chart saved: {chart_path}")



In [2]:
# if __name__ == "__main__":
#     model = torch.nn.Linear(1000, 1000)
#     optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
#     loss_fn = torch.nn.MSELoss()

#     data = torch.randn(20, 1000)
#     target = torch.randn(20, 1000)

#     profiler = CustomPyTorchProfiler(trace_name="mac_safe_profile", output_dir="prifiling_output")

#     profiler.start()

#     for epoch in range(3):
#         profiler.record_function("forward", lambda: model(data))
#         output = model(data)

#         profiler.record_function("loss", lambda: loss_fn(output, target))
#         loss = loss_fn(output, target)

#         profiler.record_function("backward", lambda: loss.backward(retain_graph=True))
#         loss.backward(retain_graph=True)

#         profiler.record_function("optimizer_step", lambda: optimizer.step())
#         optimizer.step()
#         optimizer.zero_grad()

#     profiler.stop()
#     profiler.log_summary()



In [3]:
from pytorch_profiling.pytorch_profiler import PyTorchProfiler

if __name__ == "__main__":
    model = torch.nn.Linear(1000, 1000)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    loss_fn = torch.nn.MSELoss()

    data = torch.randn(20, 1000)
    target = torch.randn(20, 1000)

    profiler = CustomPyTorchProfiler(trace_name="mac_safe_profile", output_dir="prifiling_output")
    
    with PyTorchProfiler() as profiler:
        for epoch in range(3):
            profiler.record_function("forward", lambda: model(data))
            output = model(data)
            
            profiler.record_function("loss", lambda: loss_fn(output, target))
            loss = loss_fn(output, target)
            
            profiler.record_function("backward", lambda: loss.backward(retain_graph=True))
            loss.backward(retain_graph=True)
            
            profiler.record_function("optimizer_step", lambda: optimizer.step())
            optimizer.step()
            optimizer.zero_grad()
            
    res = profiler.profiling_summary
res


STAGE:2025-04-25 00:46:54 93276:8756444 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-04-25 00:46:54 93276:8756444 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-04-25 00:46:54 93276:8756444 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


PyTorchProfilingResult(output_dir=pytorch_profiling_result_folder)

In [4]:
print(res)

PyTorch Profiling Result.

Trace Name: `default_trace`  
Total Duration: `0.0481 seconds`  
Output Directory: `pytorch_profiling_result_folder`  

- How to View:
    1. Run this in your terminal: tensorboard --logdir=pytorch_profiling_result_folder
    2. Then open [http://localhost:6006](http://localhost:6006)

- Notes:
    1. CPU-only profiling (macOS doesn't support CUDA)
    2. View the *"Profiler"* tab in TensorBoard



In [5]:
res.profiling_data

{'trace_name': 'default_trace',
 'duration': 0.04809689521789551,
 'output_dir': 'pytorch_profiling_result_folder'}